# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset


**Adaptation of the Detectron2 Tutorial File**
**Run in google colab**

# Install detectron2

In [4]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 632.3 kB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15303, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15303 (delta 9), reused 11 (delta 1), pack-reused 15275
Receiving objects: 100% (15303/15303), 6.18 MiB | 19.13 MiB/s, done.
Resolving deltas: 100% (11124/11124), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.1 MB/s eta 0:00:00
  

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.1 ; cuda:  cu121
detectron2: 0.6


In [7]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import pandas as pd
from matplotlib import pyplot as plt
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog



In [11]:
pic_list=[]
for root, dirs, files in os.walk("/content/drive/MyDrive/Obsidian/memes/defaults/images", topdown=True, onerror=None, followlinks=False):
    for filename in files:
        pic_list.append(filename)


pic_list=pic_list
pic_list

['covid_memes_3752.png',
 'covid_memes_3761.png',
 'covid_memes_2761.png',
 'covid_memes_4049.png',
 'covid_memes_2487.png',
 'covid_memes_4514.png',
 'covid_memes_4733.png',
 'covid_memes_3932.png',
 'covid_memes_5495.png',
 'covid_memes_481.png',
 'covid_memes_2832.png',
 'covid_memes_760.png',
 'covid_memes_529.png',
 'covid_memes_2849.png',
 'covid_memes_4741.png',
 'covid_memes_828.png',
 'covid_memes_4821.png',
 'covid_memes_334.png',
 'covid_memes_3017.png',
 'covid_memes_2914.png',
 'covid_memes_2448.png',
 'covid_memes_3432.png',
 'covid_memes_4638.png',
 'covid_memes_4748.png',
 'covid_memes_451.png',
 'covid_memes_4239.png',
 'covid_memes_3240.png',
 'covid_memes_2782.png',
 'covid_memes_4019.png',
 'covid_memes_620.png',
 'covid_memes_4311.png',
 'covid_memes_5639.png',
 'covid_memes_3035.png',
 'covid_memes_3313.png',
 'covid_memes_2645.png',
 'covid_memes_5649.png',
 'covid_memes_85.png',
 'covid_memes_3860.png',
 'covid_memes_585.png',
 'covid_memes_4410.png',
 'covid_me

In [12]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
import os
import cv2
import pandas as pd
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

folder_path = "/content/drive/MyDrive/Obsidian/memes/defaults/images"

# Assuming 'pic_list' is a list of image filenames
# pic_list = ...

df = []
cfg = get_cfg()

# Add project-specific configuration here
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model

# Load model weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Remove or comment out the line below to use GPU
# cfg.MODEL.DEVICE = "cpu"

predictor = DefaultPredictor(cfg)

for i, img_filename in enumerate(pic_list):
    print(f'{i} of {len(pic_list)}')
    path = os.path.join(folder_path, img_filename)
    im = cv2.imread(path)

    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    instances = outputs["instances"].to("cpu")
    classes = instances.pred_classes
    class_names = [MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[i] for i in classes]

    class_names_str = " ".join(class_names)

    tmp_dict = {
        "img": img_filename,
        "objects": class_names_str
    }

    df.append(tmp_dict)

df = pd.DataFrame(df)
df.to_csv("Detectron2_Objects.csv")

[01/10 15:45:45 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...
0 of 3544
1 of 3544
2 of 3544
3 of 3544
4 of 3544
5 of 3544
6 of 3544
7 of 3544
8 of 3544
9 of 3544
10 of 3544
11 of 3544
12 of 3544
13 of 3544
14 of 3544
15 of 3544
16 of 3544
17 of 3544
18 of 3544
19 of 3544
20 of 3544
21 of 3544
22 of 3544
23 of 3544
24 of 3544
25 of 3544
26 of 3544
27 of 3544
28 of 3544
29 of 3544
30 of 3544
31 of 3544
32 of 3544
33 of 3544
34 of 3544
35 of 3544
36 of 3544
37 of 3544
38 of 3544
39 of 3544
40 of 3544
41 of 3544
42 of 3544
43 of 3544
44 of 3544
45 of 3544
46 of 3544
47 of 3544
48 of 3544
49 of 3544
50 of 3544
51 of 3544
52 of 3544
53 of 3544
54 of 3544
55 of 3544
56 of 3544
57 of 3544
58 of 3544
59 of 3544
60 of 3544
61 of 3544
62 of 3544
63 of 3544
64 of 3544
65 of 3544
66 of 3544
67 of 3544
68 of 3544
69 of 3544
70 of 3544
71 of 3544
7

In [13]:
df

,img,objects
0,covid_memes_3752.png,
1,covid_memes_3761.png,dog person tv book person book book
2,covid_memes_2761.png,person umbrella
3,covid_memes_4049.png,person person person
4,covid_memes_2487.png,person tie
...,...,...
3539,covid_memes_1576.png,person person chair
3540,covid_memes_1342.png,cat
3541,covid_memes_1678.png,person tie
3542,covid_memes_1946.png,person person chair
